# Libraries

In [98]:
import pandas as pd
import numpy as np
import pickle
from xgboost import XGBRegressor
from sklearn import metrics

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Data Loading/Reading

In [99]:
df = pd.read_csv("data.csv")
df

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,5/2/2014 0:00,3.130000e+05,3.0,1.50,1340.0,7912.0,1.5,0.0,0.0,3,1340.0,0.0,1955.0,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA
1,5/2/2014 0:00,2.384000e+06,5.0,2.50,3650.0,9050.0,2.0,0.0,4.0,5,3370.0,280.0,1921.0,0,709 W Blaine St,Seattle,WA 98119,USA
2,5/2/2014 0:00,3.420000e+05,3.0,2.00,1930.0,11947.0,1.0,0.0,0.0,4,1930.0,0.0,1966.0,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA
3,5/2/2014 0:00,4.200000e+05,3.0,2.25,2000.0,8030.0,1.0,0.0,0.0,4,1000.0,1000.0,1963.0,0,857 170th Pl NE,Bellevue,WA 98008,USA
4,5/2/2014 0:00,5.500000e+05,4.0,2.50,1940.0,10500.0,1.0,0.0,0.0,4,1140.0,800.0,1976.0,1992,9105 170th Ave NE,Redmond,WA 98052,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,7/9/2014 0:00,3.081667e+05,3.0,1.75,1510.0,6360.0,1.0,0.0,0.0,4,1510.0,0.0,1954.0,1979,501 N 143rd St,Seattle,WA 98133,USA
4596,7/9/2014 0:00,5.343333e+05,3.0,NaN,1460.0,7573.0,2.0,0.0,0.0,3,1460.0,0.0,1983.0,2009,14855 SE 10th Pl,Bellevue,WA 98007,USA
4597,7/9/2014 0:00,4.169042e+05,3.0,NaN,3010.0,7014.0,2.0,0.0,0.0,3,3010.0,0.0,2009.0,0,759 Ilwaco Pl NE,Renton,WA 98059,USA
4598,7/10/2014 0:00,2.034000e+05,4.0,2.00,2090.0,6630.0,1.0,0.0,0.0,3,1070.0,1020.0,1974.0,0,5148 S Creston St,Seattle,WA 98178,USA


# Data Exploration

In [100]:
print(f"Number of Rows: {df.shape[0]} \nNumber of Columns: {df.shape[1]}")

Number of Rows: 4600 
Number of Columns: 18


In [101]:
df.head(2)

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,5/2/2014 0:00,313000.0,3.0,1.5,1340.0,7912.0,1.5,0.0,0.0,3,1340.0,0.0,1955.0,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA
1,5/2/2014 0:00,2384000.0,5.0,2.5,3650.0,9050.0,2.0,0.0,4.0,5,3370.0,280.0,1921.0,0,709 W Blaine St,Seattle,WA 98119,USA


In [102]:
df.tail(2)

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
4598,7/10/2014 0:00,203400.0,4.0,2.0,2090.0,6630.0,1.0,0.0,0.0,3,1070.0,1020.0,1974.0,0,5148 S Creston St,Seattle,WA 98178,USA
4599,7/10/2014 0:00,220600.0,3.0,2.5,1490.0,8102.0,2.0,0.0,0.0,4,1490.0,0.0,1990.0,0,18717 SE 258th St,Covington,WA 98042,USA


In [103]:
df.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated
count,4.597000e+03,4595.000000,4598.000000,4598.000000,4.598000e+03,4598.000000,4595.000000,4598.000000,4600.000000,4597.000000,4597.000000,4594.000000,4600.000000
mean,5.521190e+05,3.401088,2.160668,2139.498913,1.485607e+04,1.511853,0.007182,0.240757,3.451739,1827.509463,312.122036,1970.799086,808.608261
std,5.639834e+05,0.908983,0.783920,963.387024,3.589177e+04,0.537905,0.084449,0.778558,0.677230,862.138540,464.228562,29.740666,979.414536
min,0.000000e+00,0.000000,0.000000,370.000000,6.380000e+02,1.000000,0.000000,0.000000,1.000000,370.000000,0.000000,1900.000000,0.000000
25%,3.238333e+05,3.000000,1.750000,1460.000000,5.001000e+03,1.000000,0.000000,0.000000,3.000000,1190.000000,0.000000,1951.000000,0.000000
50%,4.610000e+05,3.000000,2.250000,1980.000000,7.683000e+03,1.500000,0.000000,0.000000,3.000000,1590.000000,0.000000,1976.000000,0.000000
75%,6.550000e+05,4.000000,2.500000,2620.000000,1.100375e+04,2.000000,0.000000,0.000000,4.000000,2300.000000,610.000000,1997.000000,1999.000000
max,2.659000e+07,9.000000,8.000000,13540.000000,1.074218e+06,3.500000,1.000000,4.000000,5.000000,9410.000000,4820.000000,2014.000000,2014.000000


In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           4600 non-null   object 
 1   price          4597 non-null   float64
 2   bedrooms       4595 non-null   float64
 3   bathrooms      4598 non-null   float64
 4   sqft_living    4598 non-null   float64
 5   sqft_lot       4598 non-null   float64
 6   floors         4598 non-null   float64
 7   waterfront     4595 non-null   float64
 8   view           4598 non-null   float64
 9   condition      4600 non-null   int64  
 10  sqft_above     4597 non-null   float64
 11  sqft_basement  4597 non-null   float64
 12  yr_built       4594 non-null   float64
 13  yr_renovated   4600 non-null   int64  
 14  street         4597 non-null   object 
 15  city           4596 non-null   object 
 16  statezip       4597 non-null   object 
 17  country        4594 non-null   object 
dtypes: float

In [105]:
print("-- Attributes in Data --")
for cols in df.columns:
    print(cols)

-- Attributes in Data --
date
price
bedrooms
bathrooms
sqft_living
sqft_lot
floors
waterfront
view
condition
sqft_above
sqft_basement
yr_built
yr_renovated
street
city
statezip
country


In [106]:
print("-- Number of instances in Data --")
print(df.count())

-- Number of instances in Data --
date             4600
price            4597
bedrooms         4595
bathrooms        4598
sqft_living      4598
sqft_lot         4598
floors           4598
waterfront       4595
view             4598
condition        4600
sqft_above       4597
sqft_basement    4597
yr_built         4594
yr_renovated     4600
street           4597
city             4596
statezip         4597
country          4594
dtype: int64


In [107]:
df['city'].unique()

array(['Shoreline', 'Seattle', 'Kent', 'Bellevue', 'Redmond',
       'Maple Valley', 'North Bend', 'Lake Forest Park', 'Sammamish',
       'Auburn', 'Des Moines', 'Bothell', 'Federal Way', 'Kirkland',
       'Issaquah', 'Woodinville', 'Normandy Park', 'Fall City', 'Renton',
       'Carnation', 'Snoqualmie', 'Duvall', 'Burien', 'Covington',
       'Inglewood-Finn Hill', 'Kenmore', 'Newcastle', 'Mercer Island',
       'Black Diamond', 'Ravensdale', 'Clyde Hill', 'Algona', 'Skykomish',
       'Tukwila', 'Vashon', 'Yarrow Point', 'SeaTac', 'Medina',
       'Enumclaw', 'Snoqualmie Pass', 'Pacific', 'Beaux Arts Village',
       'Preston', 'Milton', nan], dtype=object)

In [108]:
df.nunique()

date               70
price            1738
bedrooms           10
bathrooms          26
sqft_living       566
sqft_lot         3112
floors              6
waterfront          2
view                5
condition           5
sqft_above        511
sqft_basement     207
yr_built          115
yr_renovated       60
street           4522
city               44
statezip           77
country             1
dtype: int64

In [109]:
print("-- Number of Null Values in Data --")
print(df.isnull().sum())

-- Number of Null Values in Data --
date             0
price            3
bedrooms         5
bathrooms        2
sqft_living      2
sqft_lot         2
floors           2
waterfront       5
view             2
condition        0
sqft_above       3
sqft_basement    3
yr_built         6
yr_renovated     0
street           3
city             4
statezip         3
country          6
dtype: int64


# Data Pre-Processing

In [110]:
df = df.drop('date', axis=1)

In [111]:
df.head(2)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,313000.0,3.0,1.5,1340.0,7912.0,1.5,0.0,0.0,3,1340.0,0.0,1955.0,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA
1,2384000.0,5.0,2.5,3650.0,9050.0,2.0,0.0,4.0,5,3370.0,280.0,1921.0,0,709 W Blaine St,Seattle,WA 98119,USA


In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          4597 non-null   float64
 1   bedrooms       4595 non-null   float64
 2   bathrooms      4598 non-null   float64
 3   sqft_living    4598 non-null   float64
 4   sqft_lot       4598 non-null   float64
 5   floors         4598 non-null   float64
 6   waterfront     4595 non-null   float64
 7   view           4598 non-null   float64
 8   condition      4600 non-null   int64  
 9   sqft_above     4597 non-null   float64
 10  sqft_basement  4597 non-null   float64
 11  yr_built       4594 non-null   float64
 12  yr_renovated   4600 non-null   int64  
 13  street         4597 non-null   object 
 14  city           4596 non-null   object 
 15  statezip       4597 non-null   object 
 16  country        4594 non-null   object 
dtypes: float64(11), int64(2), object(4)
memory usage: 61

In [113]:
df['street'].mode()[0]

'2520 Mulberry Walk NE'

In [114]:
# df['street'] = df['street'].fillna(df['street'].mode()[0])
def fillNaObjMode(cols):
    for i in cols:
        df[i] = df[i].fillna(df[i].mode()[0])

columns = ['street','city','statezip','country']
fillNaObjMode(columns)

In [115]:
def fillNaIntMode(cols):
    for i in cols:
        df[i] = df[i].fillna(df[i].mode()[0])

columns = ['bedrooms','bathrooms','floors','waterfront','view','yr_built']
fillNaIntMode(columns)

In [116]:
def fillNaFloat(cols):
    for i in cols:
        df[i] = df[i].fillna(df[i].mean())

columns = ['price','sqft_living','sqft_lot','sqft_above','sqft_basement']
fillNaFloat(columns)

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          4600 non-null   float64
 1   bedrooms       4600 non-null   float64
 2   bathrooms      4600 non-null   float64
 3   sqft_living    4600 non-null   float64
 4   sqft_lot       4600 non-null   float64
 5   floors         4600 non-null   float64
 6   waterfront     4600 non-null   float64
 7   view           4600 non-null   float64
 8   condition      4600 non-null   int64  
 9   sqft_above     4600 non-null   float64
 10  sqft_basement  4600 non-null   float64
 11  yr_built       4600 non-null   float64
 12  yr_renovated   4600 non-null   int64  
 13  street         4600 non-null   object 
 14  city           4600 non-null   object 
 15  statezip       4600 non-null   object 
 16  country        4600 non-null   object 
dtypes: float64(11), int64(2), object(4)
memory usage: 61

In [118]:
# df['price'] = df['price'].astype('int64')

def convertFloatintoInt(cols):
    for i in cols:
        df[i] = df[i].astype('int64')

columns = ['bedrooms','bathrooms','floors','waterfront','view','yr_built','price','sqft_living','sqft_lot','sqft_above','sqft_basement']
convertFloatintoInt(columns)

In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   price          4600 non-null   int64 
 1   bedrooms       4600 non-null   int64 
 2   bathrooms      4600 non-null   int64 
 3   sqft_living    4600 non-null   int64 
 4   sqft_lot       4600 non-null   int64 
 5   floors         4600 non-null   int64 
 6   waterfront     4600 non-null   int64 
 7   view           4600 non-null   int64 
 8   condition      4600 non-null   int64 
 9   sqft_above     4600 non-null   int64 
 10  sqft_basement  4600 non-null   int64 
 11  yr_built       4600 non-null   int64 
 12  yr_renovated   4600 non-null   int64 
 13  street         4600 non-null   object
 14  city           4600 non-null   object
 15  statezip       4600 non-null   object
 16  country        4600 non-null   object
dtypes: int64(13), object(4)
memory usage: 611.1+ KB


In [120]:
df['street'].nunique()

4522

In [121]:
df['country'].nunique()

1

In [122]:
df['city'].nunique()

44

In [123]:
df['statezip'].nunique()

77

In [124]:
df = df.drop('street',axis=1)
df = df.drop('country',axis=1)

In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   price          4600 non-null   int64 
 1   bedrooms       4600 non-null   int64 
 2   bathrooms      4600 non-null   int64 
 3   sqft_living    4600 non-null   int64 
 4   sqft_lot       4600 non-null   int64 
 5   floors         4600 non-null   int64 
 6   waterfront     4600 non-null   int64 
 7   view           4600 non-null   int64 
 8   condition      4600 non-null   int64 
 9   sqft_above     4600 non-null   int64 
 10  sqft_basement  4600 non-null   int64 
 11  yr_built       4600 non-null   int64 
 12  yr_renovated   4600 non-null   int64 
 13  city           4600 non-null   object
 14  statezip       4600 non-null   object
dtypes: int64(13), object(2)
memory usage: 539.2+ KB


In [126]:
def dataEncoder(cols):
    for i in cols:
        dataLabelEncoder = LabelEncoder()
        df[i] = dataLabelEncoder.fit_transform(df[i])

columns = ['city','statezip']
dataEncoder(columns)

In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   price          4600 non-null   int64
 1   bedrooms       4600 non-null   int64
 2   bathrooms      4600 non-null   int64
 3   sqft_living    4600 non-null   int64
 4   sqft_lot       4600 non-null   int64
 5   floors         4600 non-null   int64
 6   waterfront     4600 non-null   int64
 7   view           4600 non-null   int64
 8   condition      4600 non-null   int64
 9   sqft_above     4600 non-null   int64
 10  sqft_basement  4600 non-null   int64
 11  yr_built       4600 non-null   int64
 12  yr_renovated   4600 non-null   int64
 13  city           4600 non-null   int32
 14  statezip       4600 non-null   int32
dtypes: int32(2), int64(13)
memory usage: 503.3 KB


In [128]:
df.head(2)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,city,statezip
0,313000,3,1,1340,7912,1,0,0,3,1340,0,1955,2005,36,62
1,2384000,5,2,3650,9050,2,0,4,5,3370,280,1921,0,35,58


In [129]:
df.to_csv(r'encoded-data.csv', index = False, header = True)

# Train-Test Split

In [132]:
X = df.drop(columns='bedrooms', axis=1)
Y = df['bedrooms']

In [133]:
print(X)

        price  bathrooms  sqft_living  sqft_lot  floors  waterfront  view  \
0      313000          1         1340      7912       1           0     0   
1     2384000          2         3650      9050       2           0     4   
2      342000          2         1930     11947       1           0     0   
3      420000          2         2000      8030       1           0     0   
4      550000          2         1940     10500       1           0     0   
...       ...        ...          ...       ...     ...         ...   ...   
4595   308166          1         1510      6360       1           0     0   
4596   534333          2         1460      7573       2           0     0   
4597   416904          2         3010      7014       2           0     0   
4598   203400          2         2090      6630       1           0     0   
4599   220600          2         1490      8102       2           0     0   

      condition  sqft_above  sqft_basement  yr_built  yr_renovated  city  \

In [134]:
print(Y)

0       3
1       5
2       3
3       3
4       4
       ..
4595    3
4596    3
4597    3
4598    4
4599    3
Name: bedrooms, Length: 4600, dtype: int64


In [135]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [136]:
print(X.shape, X_train.shape, X_test.shape)

(4600, 14) (3680, 14) (920, 14)


# Model Apply / Classifier Application

In [94]:
regressor = XGBRegressor()

In [137]:
regressor.fit(X_train, Y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [138]:
training_data_prediction = regressor.predict(X_train)

In [139]:
r2_train = metrics.r2_score(Y_train, training_data_prediction)

In [140]:
print('R Squared value = ', r2_train)

R Squared value =  0.9391363457061632


In [141]:
test_data_prediction = regressor.predict(X_test)

In [142]:
r2_test = metrics.r2_score(Y_test, test_data_prediction)

In [143]:
print('R Squared value = ', r2_test)

R Squared value =  0.3591629892499115
